In [1]:
import os
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from timezonefinder import TimezoneFinder
from pytz import timezone

In [10]:
df_log = pd.read_csv('../../01_data/02_preprocess/stocks.csv')
#Calculating the daily change, removing what seems to be unnecessary columns
df_log['change_pct'] = df_log['change'] *100/ df_log['open']
# Compute bounds (mean & std) per Date across all tickers
bounds = df_log.groupby('date')['change_pct'].agg(['mean', 'std']).reset_index()
df_log = df_log.drop(columns=['Unnamed: 0'])
df_log.head()

,ticker,date,open,change,change_pct
0,NYA,2008-08-01,8438.709961,-59.559570,-0.705790
1,IXIC,2008-08-01,2326.830078,-15.870117,-0.682049
2,FTSE,2008-08-01,5411.899902,-57.199707,-1.056925
3,NSEI,2008-08-01,4331.600098,81.949707,1.891904
4,BSESN,2008-08-01,14064.259766,592.430664,4.212313


In [ ]:
# Ensure the output directory exists
output_dir = '../../01_data/03_analysis/'
os.makedirs(output_dir, exist_ok=True)

# Read market and earthquake datasets
df_markets = pd.read_csv('../../01_data/02_preprocess/markets_info.csv')
df_eq = pd.read_csv('../../01_data/02_preprocess/eq_2008-23.csv')  # Earthquakes ≥ 6 from Aug 1, 2008 - end of 2009

# Convert date columns to datetime
df_eq['date'] = pd.to_datetime(df_eq['date'], format='mixed')

# Initialize TimezoneFinder
tf = TimezoneFinder()


# Process each ticker in the markets.csv file
for _, market_row in df_markets.iterrows():
    
    # LOGISTIC REGRESSION
    ticker = market_row['ticker']
    # Filter data for the current ticker
    ticker_data = df_log[df_log['ticker'] == ticker][['date', 'change_pct']]

    # Merge ticker data with precomputed bounds
    df_alt_ticker = pd.merge(ticker_data, bounds, on='date')

    # Vectorized effect classification using np.select
    conditions = [
    (df_alt_ticker['mean'] - df_alt_ticker['std'] <= df_alt_ticker['change_pct']),
    
    (df_alt_ticker['mean'] - 2 * df_alt_ticker['std'] <= df_alt_ticker['change_pct']) & 
    (df_alt_ticker['change_pct'] < df_alt_ticker['mean'] - df_alt_ticker['std']),
    
    (df_alt_ticker['mean'] - 3 * df_alt_ticker['std'] <= df_alt_ticker['change_pct']) & 
    (df_alt_ticker['change_pct'] < df_alt_ticker['mean'] - 2 * df_alt_ticker['std'])
]


    choices = [0, 1, 2]
    df_alt_ticker['effect'] = np.select(conditions, choices, default=3)
    # Convert 'Date' column in df_alt_ticker to datetime without time
    df_alt_ticker['date'] = pd.to_datetime(df_alt_ticker['date'], errors='coerce').dt.date    
#END OF LOGISTIC


    ticker = market_row['ticker']
    
    # Convert latitude and longitude to float
    latitude = float(market_row['Latitude'])
    longitude = float(market_row['Longitude'])

    # Convert close time to datetime.time object
    close_time = pd.to_datetime(market_row['close'], format='%H:%M', errors='coerce').time()

    # Find the time zone based on latitude and longitude
    timezone_str = tf.timezone_at(lat=latitude, lng=longitude)
    tz = timezone(timezone_str)

    # Read the stock market data for the current ticker
    df_ticker = pd.read_csv(f'../../01_data/02_preprocess/01_index/{ticker}.csv')
    #df_ticker.rename(columns={'Date': 'date'}, inplace=True)  # Rename 'Date' column to 'date'

    # Convert df_ticker date column to the respective time zone
    df_ticker['date'] = pd.to_datetime(df_ticker['date']).dt.tz_localize(tz)

    # Normalize the earthquake data to the close time of the current ticker
    if df_eq['date'].dt.tz is None:
        # If the column is timezone-naive, localize it to UTC first
        df_eq['date_close'] = df_eq['date'].dt.tz_localize('UTC').dt.tz_convert(tz).dt.normalize() + pd.Timedelta(hours=close_time.hour, minutes=close_time.minute)
    else:
        # If the column is already timezone-aware, convert it directly
        df_eq['date_close'] = df_eq['date'].dt.tz_convert(tz).dt.normalize() + pd.Timedelta(hours=close_time.hour, minutes=close_time.minute)

    # Define the coordinates for the current market
    market_coords = (latitude, longitude)

    # Initialize lists for new columns
    num_list, sum_list, max_mag, max_sig, min_depth, min_dist_list, sum_tsunami = [], [], [], [], [], [], []

    # Process each row in df_ticker
    for i, ticker_row in df_ticker.iterrows():
        curr_date = ticker_row['date']
        prev_date = df_ticker.iloc[i - 1]['date'] if i > 0 else None

        # Define time window: after previous day's close and before current day's close
        if prev_date is not None:
            eq_filtered = df_eq[(df_eq['date'] > prev_date + pd.Timedelta(hours=close_time.hour, minutes=close_time.minute)) & 
                               (df_eq['date'] <= curr_date + pd.Timedelta(hours=close_time.hour, minutes=close_time.minute))]
        else:
            eq_filtered = df_eq[df_eq['date'] <= curr_date + pd.Timedelta(hours=close_time.hour, minutes=close_time.minute)]

        # Compute required values
        num_list.append(len(eq_filtered))
        sum_list.append(eq_filtered['magnitudo'].sum() if not eq_filtered.empty else np.nan)
        max_mag.append(eq_filtered['magnitudo'].max() if not eq_filtered.empty else np.nan)
        max_sig.append(eq_filtered['significance'].max() if not eq_filtered.empty else np.nan)
        min_depth.append(eq_filtered['depth'].min() if not eq_filtered.empty else np.nan)
        sum_tsunami.append(eq_filtered['tsunami'].sum() if not eq_filtered.empty else np.nan)
        
        # Compute distances from the market's location
        if not eq_filtered.empty:
            distances = eq_filtered.apply(lambda row: geodesic((row['latitude'], row['longitude']), market_coords).km, axis=1)
            min_dist_list.append(distances.min())
        else:
            min_dist_list.append(np.nan)      
        
    # Create the combined df
    df = df_ticker.copy()
    df['num'] = num_list
    df['sum'] = sum_list
    df['max_mag'] = max_mag
    df['max_sig'] = max_sig
    df['min_depth'] = min_depth
    df['min_dist'] = min_dist_list
    df['tsunami'] = sum_tsunami
    # Convert 'date' column in df to datetime and discard the time part
    df['date'] = pd.to_datetime(df['date']).dt.date
    
    
    # Merge the two DataFrames based on the Date and date columns
    merged_df = pd.merge(df_alt_ticker, df, left_on='date', right_on='date', how='inner')    
    merged_df = merged_df.drop(columns=['mean', 'std'])
    # Reorder columns
    merged_df = merged_df[['date', 'change', 'change_pct', 'effect', 'num', 'sum', 'max_mag', 'max_sig', 'min_depth', 'min_dist', 'tsunami']]


    # Save the dataframe to a CSV file named according to the ticker
    output_file = os.path.join(output_dir, f'{ticker}.csv')
    merged_df.to_csv(output_file, index=False)
    print(f"Saved data for ticker {ticker} to {output_file}")

Saved data for ticker NYA to ../../01_data/03_analysis/NYA.csv
Saved data for ticker IXIC to ../../01_data/03_analysis/IXIC.csv
Saved data for ticker FTSE to ../../01_data/03_analysis/FTSE.csv
Saved data for ticker NSEI to ../../01_data/03_analysis/NSEI.csv
Saved data for ticker BSESN to ../../01_data/03_analysis/BSESN.csv
Saved data for ticker N225 to ../../01_data/03_analysis/N225.csv
Saved data for ticker 000001SS to ../../01_data/03_analysis/000001SS.csv
Saved data for ticker N100 to ../../01_data/03_analysis/N100.csv
Saved data for ticker DJI to ../../01_data/03_analysis/DJI.csv
Saved data for ticker GSPC to ../../01_data/03_analysis/GSPC.csv
